## Python Notebook to train XGBoost model on both the original and augmented dataset

In [70]:
#Import necessary modules
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import nltk
import xgboost as xgb

## Configure NLTK if applicable

In [ ]:
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('vader_lexicon')

In [71]:
# obtained from https://gist.github.com/susanli2016/d35def30b99f0e2f56c0e01e19ad0878
def gettop_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [84]:
def get_len(row):
    return len(row['Text'])

def get_len_aug(row):
    return len(row['text'])

In [73]:
# Perform feature engineering on the dataset
def feature_engineering(dataset, aug=False):
    if (aug):
        dataset['los'] = dataset.apply(get_len_aug, axis=1)
    else:
        dataset['los'] = dataset.apply(get_len, axis=1)
    return dataset

In [85]:
train_original = pd.read_csv('dataset/fulltrain.csv')
train_augment = pd.read_csv('dataset/merged_final_df_with_topics.csv')
X = feature_engineering(train_original)['los'].to_frame()

y = train_original['Label']
# Comment this out if we are just using the original dataset
# y = train_augment['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
train_augment[:10]

,label,text,has_swear_word,severity,processed_text,topic
0,1,"A little less than a decade ago, hockey fans w...",False,0.0,"['little', 'less', 'decade', 'ago', 'hockey', ...",0
1,1,The writers of the HBO series The Sopranos too...,False,0.0,"['writers', 'hbo', 'series', 'sopranos', 'took...",0
2,1,Despite claims from the TV news outlet to offe...,False,0.0,"['despite', 'claims', 'tv', 'news', 'outlet', ...",0
3,1,After receiving 'subpar' service and experienc...,False,0.0,"['receiving', 'subpar', 'service', 'experienci...",4
4,1,After watching his beloved Seattle Mariners pr...,False,0.0,"['watching', 'beloved', 'seattle', 'mariners',...",0
5,1,"At a cafeteria-table press conference Monday, ...",False,0.0,"['cafeteriatable', 'press', 'conference', 'mon...",4
6,1,Stunned shock and dismay were just a few of th...,False,0.0,"['stunned', 'shock', 'dismay', 'reactions', 'b...",0
7,1,"Speaking with reporters before a game Monday, ...",True,1.0,"['speaking', 'reporters', 'game', 'monday', 'l...",0
8,1,Sports journalists and television crews were p...,False,0.0,"['sports', 'journalists', 'television', 'crews...",0
9,1,"SALEM, VAF;or the eighth straight world-histor...",False,0.0,"['salem', 'vafor', 'eighth', 'straight', 'worl...",0


In [78]:
#Run this if you need to modify X_train again for some reason
# X_train = train_augment['topic']
print(X_train)

        los
58515   844
28095  9019
2081   1077
41524   970
53155  7952
...     ...
54343   868
38158  2630
860    4033
15795  5460
56422   828

[47836 rows x 1 columns]


In [86]:
model = GradientBoostingClassifier() # not xgboost, but just to test that a model works
X_train_formatted = np.array(X_train).reshape(-1, 1)
# X_train_formatted = np.array(X_train)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.5265790920526502

### Notes

As a baseline for the "random" f1 score for the original dataset, I trained the XGBoost model on an X_train that was just the length of the text string. This f1 score turned out to be 0.5266, which is surprisingly high and not much less than the f1 score in the paper.
Doing the same for the augmented dataset yields an F1 score of 0.4983, which is lower but is expected as the length of the text string should theoretically have zero correlation with the label.